# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint



### Not for Grading

## Learning Objective

The objective of this experiment is to understand the influence of tuning hyperparameters to avoid Overfitting.


### Dataset Decription:
The dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

The datasets consists of several medical predictor variables and one target variable, Outcome.

    Preg: Number of times pregnant
    Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
    BloodPressure: Diastolic blood pressure (mm Hg)
    SkinThickness: Triceps skin fold thickness (mm)
    Insulin: 2-Hour serum insulin (mu U/ml)
    BMI: Body mass index (weight in kg/(height in m)^2)
    DiabetesPedigreeFunction: Diabetes pedigree function
    Age: Age (years)
    Outcome: Class variable (0 or 1)


In [ ]:
! wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/diabetes.csv


### Importing Required Packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

### Prepare the model

In [ ]:
diabetes_data = pd.read_csv("/content/diabetes.csv")
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Extracting features and labels from the diabetes data

In [ ]:
train_features = diabetes_data.iloc[:,:-1].values
labels = diabetes_data.iloc[:,-1].values
train_features.shape, labels.shape

((768, 8), (768,))

Calculate the training accuracy

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(train_features, labels, test_size = 0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(614, 8) (154, 8) (614,) (154,)


In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train,y_train)
train_pred = decision_tree.predict(X_train)
train_acc = accuracy_score(train_pred, y_train)
print("Training accuracy: ", train_acc)

Training accuracy:  1.0


Calculate the testing accuracy

In [ ]:
# Verify the accuracy of the test data
test_pred = decision_tree.predict(X_test)
test_acc = accuracy_score(test_pred, y_test)
print("Testing accuracy", test_acc) 

Testing accuracy 0.7597402597402597



Although the training accuracy was very high, when we run the model on an unseen test dataset, we are getting less accuracy. It means that the model is not generalizing well to unseen data. This is known as **Overfitting**.




### Overfitting 


A model that has learned the noise instead of (or alongwith) the signal is considered “overfit” because it fits the training dataset very well but has a poor fit with unseen test datasets.

When the tree grows deeper, it starts capturing random fluctuations (noise) in the data. This is one of the root causes of overfitting in decision trees. So, if the model is overfitting, reducing the max_depth value is one way to combat overfitting.

To avoid overfitting, we should tune the parameters like max_depth in a decision tree appropriately.  


Find the the train and test accuracy at different depths and identify the point at which the test accuracy starts decreasing. In the final decision tree model, we can specify the max_depth value after which the testing accuracy starts decreasing. This will allow the model to generalize well to further unseen data.


In [ ]:
# Calculating accuracy of train and test data
depths = np.arange(1,15)
train_acc, test_acc = [], []

for i in depths:
  decision_tree = DecisionTreeClassifier(max_depth=i, random_state=1)
  decision_tree.fit(X_train,y_train)
  y_train_pred = decision_tree.predict(X_train)
  train_acc.append(accuracy_score(y_train_pred, y_train))

  # Test accuracy calculation
  y_test_pred = decision_tree.predict(X_test)
  test_acc.append(accuracy_score(y_test_pred, y_test))

acc_data = pd.DataFrame({'depth': depths, 'Train accuracy': train_acc, 'Test Accuracy': test_acc})
acc_data.set_index('depth', inplace=True)
acc_data

,Train accuracy,Test Accuracy
depth,,
1,0.734528,0.740260
2,0.771987,0.772727
3,0.776873,0.759740
4,0.798046,0.694805
5,0.842020,0.798701
6,0.866450,0.740260
7,0.910423,0.753247
8,0.942997,0.766234
9,0.970684,0.753247


### Apply KFold

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=42)
scores_Train = []
scores_Test = []
x_data =train_features
# i=1
for train_index, test_index in kf.split(x_data):
   
    x_train, x_test, y_train, y_test = x_data[train_index], x_data[test_index], labels[train_index], labels[test_index]
    decision_tree2 = DecisionTreeClassifier(max_depth=2)
    decision_tree2.fit(x_train, y_train)
    scores_Train.append(decision_tree2.score(x_train, y_train))
    scores_Test.append(decision_tree2.score(x_test, y_test))

#  Finally, we can average all the scores from all 5 cross validations and get an average score for Train and Test data
print("Average score of the Training set %.2f"%np.mean(scores_Train))
print("Average score of the Testing set %.2f"%np.mean(scores_Test))

Average score of the Training set 0.84
Average score of the Testing set 0.74
